In [1]:
import pandas as pd
import arcgis

# Load Data into a SpatialDataFrame

Load the data into a Pandas DataFrame from a CSV file.

In [12]:
df = pd.read_csv('./store_locations.csv', index_col='OBJECTID')
df.head()

,LOCNUM,SALESVOL,X,Y
OBJECTID,,,,
1,666990510,35495,-121.8430,36.6210
2,653371815,35495,-121.8112,36.6676
3,423468472,35495,-121.9651,36.9753
4,511743478,35495,-121.7740,36.9154
5,404459478,52059,-122.0362,37.3231


While the coordinates for each store are contained in an X (longitude) and Y (latitude) field, the data is not yet able to be recognized spatially. We need to create a point geometry for each location in a new field so the data will be recognized as spatial. Once this is done, we also can get rid of the explicity X and Y fields, since the location is now stored in the SHAPE field.

In [13]:
df['SHAPE'] = df.apply(lambda row: arcgis.geometry.Point(
    {'x': row.X, 'y': row.Y, 'spatialReference': arcgis.geometry.SpatialReference({'wkid': 4326})}
), axis=1)
df = df.drop(['X', 'Y'], axis=1)
df.head()

,LOCNUM,SALESVOL,SHAPE
OBJECTID,,,
1,666990510,35495,"{'y': 36.621, 'spatialReference': {'wkid': 432..."
2,653371815,35495,"{'y': 36.6676, 'spatialReference': {'wkid': 43..."
3,423468472,35495,"{'y': 36.9753, 'spatialReference': {'wkid': 43..."
4,511743478,35495,"{'y': 36.9154, 'spatialReference': {'wkid': 43..."
5,404459478,52059,"{'y': 37.3231, 'spatialReference': {'wkid': 43..."


Now, with the location data stored as point geometry, we can create a SpatialDataFrame with the store locations so the data will now be recognized as a spatial table.

In [14]:
sdf = arcgis.features.SpatialDataFrame(df)
sdf.head()

,LOCNUM,SALESVOL,SHAPE
OBJECTID,,,
1,666990510,35495,"{'y': 36.621, 'spatialReference': {'wkid': 432..."
2,653371815,35495,"{'y': 36.6676, 'spatialReference': {'wkid': 43..."
3,423468472,35495,"{'y': 36.9753, 'spatialReference': {'wkid': 43..."
4,511743478,35495,"{'y': 36.9154, 'spatialReference': {'wkid': 43..."
5,404459478,52059,"{'y': 37.3231, 'spatialReference': {'wkid': 43..."


Convert the SpatailDataFrame to a FeatureSet to use as input for creating drive time trade areas.

In [15]:
#fs = arcgis.features.SpatialDataFrame(df).to_featureset  # chaining all in one line
fs = sdf.head().to_featureset  # for now, just testing with first five records
fs

<bound method SpatialDataFrame.to_featureset of              LOCNUM  SALESVOL  \
OBJECTID                        
1         666990510     35495   
2         653371815     35495   
3         423468472     35495   
4         511743478     35495   
5         404459478     52059   

                                                      SHAPE  
OBJECTID                                                     
1         {'y': 36.621, 'spatialReference': {'wkid': 432...  
2         {'y': 36.6676, 'spatialReference': {'wkid': 43...  
3         {'y': 36.9753, 'spatialReference': {'wkid': 43...  
4         {'y': 36.9154, 'spatialReference': {'wkid': 43...  
5         {'y': 37.3231, 'spatialReference': {'wkid': 43...  >

# Create GIS Instance

In [6]:
from getpass import getpass

gis_coldbrew = arcgis.gis.GIS(
    url='http://portal.coldbrew.esri.com/portal',
    username='headless', 
    password=getpass('Please enter the headless password: ')
)

Please enter the headless password: ········


In [7]:
service_area_layer = arcgis.network.ServiceAreaLayer(
    url=gis_coldbrew.properties.helperServices.serviceArea.url, 
    gis=gis_coldbrew
)

In [9]:
travel_modes = service_area_layer.retrieve_travel_modes()
travel_mode_drive = [t for t in travel_modes['supportedTravelModes'] if t['name'] == 'Driving Time'][0]
travel_mode_drive

{'attributeParameterValues': [{'attributeName': 'Avoid Unpaved Roads',
   'parameterName': 'Restriction Usage',
   'value': 'AVOID_HIGH'},
  {'attributeName': 'Avoid Private Roads',
   'parameterName': 'Restriction Usage',
   'value': 'AVOID_MEDIUM'},
  {'attributeName': 'Driving an Automobile',
   'parameterName': 'Restriction Usage',
   'value': 'PROHIBITED'},
  {'attributeName': 'Through Traffic Prohibited',
   'parameterName': 'Restriction Usage',
   'value': 'AVOID_HIGH'},
  {'attributeName': 'Roads Under Construction Prohibited',
   'parameterName': 'Restriction Usage',
   'value': 'PROHIBITED'},
  {'attributeName': 'Avoid Gates',
   'parameterName': 'Restriction Usage',
   'value': 'AVOID_MEDIUM'},
  {'attributeName': 'Avoid Express Lanes',
   'parameterName': 'Restriction Usage',
   'value': 'PROHIBITED'},
  {'attributeName': 'Avoid Carpool Roads',
   'parameterName': 'Restriction Usage',
   'value': 'PROHIBITED'}],
 'description': '',
 'distanceAttributeName': 'Kilometers',
 '

In [10]:
result = service_area_layer.solve_service_area(fs, default_breaks=[8])

Invalid or missing input parameters.


RuntimeError: Invalid or missing input parameters.
(Error Code: 400)